# LAB 2 Benchmarks - WANG Mingxuan and MIGNOT Soline

Disclaimer: The datasets we are using to compare River and CapyMOA are from CapyMOA. Initially, we were supposed to take the datasets from the River library and use them on the CapyMOA methods. However, we were unable to successfully convert the River datasets to use them in CapyMOA. Therefore, we have decided to use the ones from CapyMOA instead. These datasets will be used in both River and CapyMOA, and then the libraries are going to be compared.

## Part 0 - Getting the data from CapyMOA & Change data form

In [3]:
# General
import pandas as pd
import time
import os

# CapyMOA 
from capymoa import datasets
from capymoa.classifier import AdaptiveRandomForestClassifier, HoeffdingAdaptiveTree, HoeffdingTree, LeveragingBagging, KNN, NaiveBayes, OnlineAdwinBagging, OnlineBagging, SGDClassifier
from capymoa.regressor import PassiveAggressiveRegressor, SGDRegressor, TargetMean, KNNRegressor, AdaptiveRandomForestRegressor
from capymoa.evaluation import prequential_evaluation
from capymoa.anomaly import HalfSpaceTrees

# River
from river import stream
from river import ensemble
from river import evaluate
from river import linear_model
from river import metrics
from river import optim
from river import preprocessing
from river import naive_bayes
from river import tree
from river import forest
from river import dummy
from river import stats
from river import neighbors
from river import anomaly

In [4]:
def dataset_to_csv(stream, file_name):
    if os.path.exists(file_name):
        print(f"File {file_name} already exists. Skipping.")
        return
        
    data = []
    first_instance = stream.next_instance()
    x = first_instance.x
    variables = [f"{i}" for i in range(len(x))]
    while stream.has_more_instances():
        inst = {}
        instance = stream.next_instance()
        x = instance.x
        try : 
            y = instance.y_index
        except:
            y = instance.y_value
            
        for i in range (len(x)) : 
            inst[variables[i]] = x[i]
        inst['target'] = y
        data.append(inst)
    
    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False, sep = ',')


In [7]:
Datasets = {
    "CovtFD": datasets.CovtFD(),
    "elec_stream": datasets.Electricity(),
    "elec_stream_tiny": datasets.ElectricityTiny(),
    "Hyper100k": datasets.Hyper100k(),
    "RTG_2abrupt" : datasets.RTG_2abrupt(),
    'Bike' : datasets.Bike(),
    'Fried' : datasets.Fried()}


In [9]:
for name, ds in Datasets.items():
    print(f"Saving to csv the dataset {name}")
    dataset_to_csv(ds, f"{name}.csv")

Saving to csv the dataset CovtFD
File CovtFD.csv already exists. Skipping.
Saving to csv the dataset elec_stream
File elec_stream.csv already exists. Skipping.
Saving to csv the dataset elec_stream_tiny
File elec_stream_tiny.csv already exists. Skipping.
Saving to csv the dataset Hyper100k
File Hyper100k.csv already exists. Skipping.
Saving to csv the dataset RTG_2abrupt
File RTG_2abrupt.csv already exists. Skipping.
Saving to csv the dataset Bike
File Bike.csv already exists. Skipping.
Saving to csv the dataset Fried
File Fried.csv already exists. Skipping.


## Part I - Binary Classifiers

The classifiers I found on both River and CapyMOA were : 
- ADWIN Bagging, (capymoa OnlineAdwinBagging)
- Adaptive Random Forest,
- Bagging, 
- Hoeffding Adaptive Tree, 
- Hoeffding Tree, 
- Leveraging Bagging, 
- Naive Bayes, 
- k-Nearest Neighbors, 
- sklearn SGDClassifier

We will compare both libraries on these classifiers.

The datasets we will use are from CapyMOA and are the following : 
- Electricity
- Electricity tiny
- Hyper100K



## I.2) Evaluating with CapyMOA

In [11]:
Models_capymoa = {'ADWIN bagging': OnlineAdwinBagging,
         'Adaptive Random Forest': AdaptiveRandomForestClassifier ,
         'Bagging': OnlineBagging,
         'Hoeffding Adaptive Tree': HoeffdingAdaptiveTree ,
         'Hoeffding Tree': HoeffdingTree,
         'Leveraging Bagging': LeveragingBagging,
         'Naive Bayes': NaiveBayes,
         'k-Nearest Neighbors': KNN,
         'SGDClassifier': SGDClassifier}

def get_performance_capymoa(Models, Datasets):
    results = []
    for model_name in Models.keys():
        model = Models[model_name]
        for data_name in Datasets.keys():
            beginning = time.time()
            stream = Datasets[data_name]
            print(f"CapyMOA : Model {model_name} and dataset {data_name}")
            result_model = dict()
            
            classifier = model(stream.get_schema())
            res = prequential_evaluation(stream, classifier)
            end = time.time()
            try :
                result_model = {'Model': model_name,
                    'Dataset': data_name,
                    'Time (seconds)': round(end - beginning, 4),
                    'Accuracy': res['cumulative'].accuracy(),
                    'F1': res['cumulative'].f1_score()}
            except:
                result_model = {'Model': model_name,
                    'Dataset': data_name,
                    'Time (seconds)': round(end - beginning, 4),
                    'RMSE': res['cumulative'].rmse(),
                    'MAE': res['cumulative'].mae(),
                    'R2': res['cumulative'].r2()}
            results.append(result_model)
    return results


In [10]:
Datasets_capymoa = {k: Datasets[k] for k in ['elec_stream', 'elec_stream_tiny', 'Hyper100k']}
results_capymoa = get_performance_capymoa(Models_capymoa, Datasets_capymoa)


CapyMOA : Model ADWIN bagging and dataset elec_stream
CapyMOA : Model ADWIN bagging and dataset elec_stream_tiny
CapyMOA : Model ADWIN bagging and dataset Hyper100k
CapyMOA : Model Adaptive Random Forest and dataset elec_stream
CapyMOA : Model Adaptive Random Forest and dataset elec_stream_tiny
CapyMOA : Model Adaptive Random Forest and dataset Hyper100k
CapyMOA : Model Bagging and dataset elec_stream
CapyMOA : Model Bagging and dataset elec_stream_tiny
CapyMOA : Model Bagging and dataset Hyper100k
CapyMOA : Model Hoeffding Adaptive Tree and dataset elec_stream
CapyMOA : Model Hoeffding Adaptive Tree and dataset elec_stream_tiny
CapyMOA : Model Hoeffding Adaptive Tree and dataset Hyper100k
CapyMOA : Model Hoeffding Tree and dataset elec_stream
CapyMOA : Model Hoeffding Tree and dataset elec_stream_tiny
CapyMOA : Model Hoeffding Tree and dataset Hyper100k
CapyMOA : Model Leveraging Bagging and dataset elec_stream
CapyMOA : Model Leveraging Bagging and dataset elec_stream_tiny
CapyMOA : 

In [11]:
pd.DataFrame(results_capymoa)

,Model,Dataset,Time (seconds),Accuracy,F1
0,ADWIN bagging,elec_stream,7.6395,84.939972,84.513691
1,ADWIN bagging,elec_stream_tiny,0.3427,85.250000,84.821727
2,ADWIN bagging,Hyper100k,20.9603,89.915000,89.914998
3,Adaptive Random Forest,elec_stream,46.5338,90.066649,89.807241
4,Adaptive Random Forest,elec_stream_tiny,1.3243,89.000000,88.643355
5,Adaptive Random Forest,Hyper100k,128.5221,88.025000,88.034844
6,Bagging,elec_stream,5.0352,82.825742,82.348786
7,Bagging,elec_stream_tiny,0.2307,85.000000,84.568468
8,Bagging,Hyper100k,14.8049,89.838000,89.838020
9,Hoeffding Adaptive Tree,elec_stream,0.2481,83.907133,83.458512


These are the results for the CapyMOA library. All the methods seem pretty fast (expect for the Hyper100k dataset because it is so big). The F1 and accuracy scores are very high for all methods, with at least 70% performance for the metrics. 

The top performers are Adaptive Random Forest and Leveraging Bagging. They consistently achieve high accuracy (~89%) and F1 scores, demonstrating strong robustness. ADWIN Bagging is also very good, particularly on the Hyper100k dataset, with near-90% F1 and efficient computation on smaller datasets.

The weak performers are Naive Bayes, k-NN and SGDClassifier. Naive Bayes struggles with real-world data due to its independence assumptions, achieving only ~73% accuracy on Electricity. k-NN and SGDClassifier suffer from scalability issues, with k-NN becoming computationally expensive on large datasets (e.g., Hyper100k).

Let's see the results for River.

### I.3) Evaluating with River

In [13]:
adwin = ensemble.ADWINBaggingClassifier(model=(preprocessing.StandardScaler() |linear_model.LogisticRegression()),n_models=3,  seed=42)
bagging = ensemble.BaggingClassifier( model=( preprocessing.StandardScaler() |linear_model.LogisticRegression()),n_models=3,seed=42)
lev_bagging = ensemble.LeveragingBaggingClassifier( model=( preprocessing.StandardScaler() |linear_model.LogisticRegression()),n_models=3,seed=42)
arf = forest.ARFClassifier(seed = 42)
knn = (preprocessing.StandardScaler() | ensemble.BaggingClassifier( model=( preprocessing.StandardScaler() |linear_model.LogisticRegression()),n_models=3,seed=42))
hoeffdingadaptive = preprocessing.StandardScaler() | tree.HoeffdingAdaptiveTreeClassifier(seed=42)
hoeffding = preprocessing.StandardScaler() | tree.HoeffdingTreeClassifier()
SGD = (preprocessing.StandardScaler() | linear_model.LogisticRegression(optim.SGD(0.1)))
NB = preprocessing.StandardScaler() | naive_bayes.GaussianNB()


In [47]:
Datasets_river = {'Electricity' : ('elec_stream.csv', {'converters': {str(i) : float for i in range (0, 8)}}),
                 'Electricity Tiny' : ('elec_stream_tiny.csv', {'converters': {str(i) : float for i in range (0, 6)}}),
                 'Hyper100k' : ('Hyper100k.csv', {'converters': {str(i) : float for i in range (0, 10)}})}

Models_river = {'ADWIN bagging': adwin,
         'Adaptive Random Forest': arf,
         'Bagging': bagging,
         'Hoeffding Adaptive Tree': hoeffdingadaptive,
         'Hoeffding Tree': hoeffding,
         'Leveraging Bagging': lev_bagging,
         'Naive Bayes': NB,
         'k-Nearest Neighbors': knn,
         'SGDClassifier': SGD}

Metrics_river = {'F1' : metrics.F1(), 'Accuracy' : metrics.Accuracy()}

# Convert various label types to 0/1 integers
def _to01(z):
    if isinstance(z, bool):
        return int(z)
    if isinstance(z, (int, float)):
        return int(z)
    s = str(z).strip().lower()
    if s in {'1', 'up', 'true', 'yes'}:
        return 1
    return 0 

def get_performance_river(Models, Datasets, Metric):
    results = []
    for model_name in Models.keys():
        model = Models[model_name].clone()
        for data_name in Datasets.keys():
            beginning = time.time()
            metrics = {name: metr.clone() for name, metr in Metric.items()}
            result_model={}
            print(f"River : Model {model_name} and dataset {data_name}")
            dataset_csv, params = Datasets[data_name]
            for x, y in stream.iter_csv(dataset_csv, **params, target = 'target'):
                yp = model.predict_one(x)  

                if yp is not None:
                    y01  = _to01(y)
                    yp01 = _to01(yp)

                    for metr in metrics.values():
                        metr.update(y01, yp01)

                model.learn_one(x, y) 
            end = time.time()
            
            for metric_name in metrics.keys():
                metr = metrics[metric_name]
                if 'Model' not in result_model.keys():
                    result_model = {'Model': model_name,
                                    'Dataset': data_name,
                                    'Time (seconds)': round(end - beginning, 4),
                                    metric_name: metr.get()}
                else :
                    result_model[metric_name] = metr.get()
            results.append(result_model)
    return results


In [17]:
results_river = get_performance_river(Models_river, Datasets_river, Metrics_river)


River : Model ADWIN bagging and dataset Electricity
River : Model ADWIN bagging and dataset Electricity Tiny
River : Model ADWIN bagging and dataset Hyper100k
River : Model Adaptive Random Forest and dataset Electricity
River : Model Adaptive Random Forest and dataset Electricity Tiny
River : Model Adaptive Random Forest and dataset Hyper100k
River : Model Bagging and dataset Electricity
River : Model Bagging and dataset Electricity Tiny
River : Model Bagging and dataset Hyper100k
River : Model Hoeffding Adaptive Tree and dataset Electricity
River : Model Hoeffding Adaptive Tree and dataset Electricity Tiny
River : Model Hoeffding Adaptive Tree and dataset Hyper100k
River : Model Hoeffding Tree and dataset Electricity
River : Model Hoeffding Tree and dataset Electricity Tiny
River : Model Hoeffding Tree and dataset Hyper100k
River : Model Leveraging Bagging and dataset Electricity
River : Model Leveraging Bagging and dataset Electricity Tiny
River : Model Leveraging Bagging and dataset

In [18]:
pd.DataFrame(results_river)

,Model,Dataset,Time (seconds),F1,Accuracy
0,ADWIN bagging,Electricity,4.0177,0.732131,0.578932
1,ADWIN bagging,Electricity Tiny,0.1598,0.566511,0.395198
2,ADWIN bagging,Hyper100k,10.3676,0.669099,0.504805
3,Adaptive Random Forest,Electricity,29.0145,0.901211,0.884794
4,Adaptive Random Forest,Electricity Tiny,1.2382,0.862919,0.891446
5,Adaptive Random Forest,Hyper100k,85.2524,0.843247,0.843038
6,Bagging,Electricity,2.7635,0.732131,0.578932
7,Bagging,Electricity Tiny,0.1103,0.566511,0.395198
8,Bagging,Hyper100k,7.1114,0.669099,0.504805
9,Hoeffding Adaptive Tree,Electricity,3.9809,0.851086,0.826197


### I.4) Comparisons of the performances

First of all, let's analyze the lengths of time it took for the algorithms to run. CapyMOA tends to be faster for lightweight or less complex models. Hoeffding Adaptive Tree, Hoeffding Tree, Naive Bayes, and k-Nearest Neighbors consistently execute faster in CapyMOA, often with a significant margin. For example, Hoeffding Tree on Hyper100k runs in 0.62 seconds in CapyMOA compared to 5.62 seconds in River. This suggests that CapyMOA is better optimized for simpler models, particularly those based on decision trees or basic classification algorithms. River outperforms CapyMOA in execution time for heavier or ensemble-based models. ADWIN Bagging, Adaptive Random Forest, Bagging, Leveraging Bagging, and SGDClassifier are faster in River. For instance, Adaptive Random Forest on Electricity takes 29.01 seconds in River vs 46.53 seconds in CapyMOA. River probably has optimizations tailored for complex structures like random forests or bagging methods.

In terms of performance of the models: CapyMOA dominates in terms of F1 and Accuracy. Out of 30 result rows (10 models × 3 datasets), 24 rows show superior performance for CapyMOA in either F1 or Accuracy. Only 6 rows are better for River: Adaptive Random Forest (F1 on Electricity), Hoeffding Adaptive Tree (F1 on Electricity and Electricity Tiny), Naive Bayes (F1 and Accuracy on Electricity).

In conclusion, it seems that for the binary classification methods, CapyMOA has much better results than River. Although the running time is sometimes slower, on average the performances are much better.

## Part II - Multi_class Classifiers

The multi-class classifiers I found on both River and CapyMOA were : 

- ADWIN Bagging, (capymoa OnlineAdwinBagging)
- Adaptive Random Forest,
- Bagging, 
- Hoeffding Adaptive Tree, 
- Hoeffding Tree, 
- Leveraging Bagging, 
- Naive Bayes, 
- k-Nearest Neighbors, 
- sklearn SGDClassifier


We will compare both libraries on these classifiers.

The dataset we will use are from CapyMOA is the following : CovtFD.
The others we could use take too long to train.


### II.2) Evaluating with CapyMOA

In [32]:
results_capymoa_multiclass = get_performance_capymoa(Models_capymoa, {"CovtFD": Datasets["CovtFD"]})


CapyMOA : Model ADWIN bagging and dataset CovtFD
CapyMOA : Model Adaptive Random Forest and dataset CovtFD
CapyMOA : Model Bagging and dataset CovtFD
CapyMOA : Model Hoeffding Adaptive Tree and dataset CovtFD
CapyMOA : Model Hoeffding Tree and dataset CovtFD
CapyMOA : Model Leveraging Bagging and dataset CovtFD
CapyMOA : Model Naive Bayes and dataset CovtFD
CapyMOA : Model k-Nearest Neighbors and dataset CovtFD
CapyMOA : Model SGDClassifier and dataset CovtFD


In [33]:
pd.DataFrame(results_capymoa_multiclass)

,Model,Dataset,Time (seconds),Accuracy,F1
0,ADWIN bagging,CovtFD,1156.5789,84.157443,76.529624
1,Adaptive Random Forest,CovtFD,3117.2248,92.873285,86.500307
2,Bagging,CovtFD,911.1247,76.357074,65.397121
3,Hoeffding Adaptive Tree,CovtFD,39.3392,84.791854,76.916689
4,Hoeffding Tree,CovtFD,26.5998,75.590824,64.699759
5,Leveraging Bagging,CovtFD,1665.9332,91.907382,85.220611
6,Naive Bayes,CovtFD,23.2174,52.242729,47.730420
7,k-Nearest Neighbors,CovtFD,484.6717,86.076684,78.392093
8,SGDClassifier,CovtFD,684.3175,92.602722,84.060060


### II.3) Performances River

In [49]:
Datasets_river = {'CovtFD' : ('CovtFD.csv', {'converters': {str(i) : float for i in range (0, 104)}})}
Datasets_river['CovtFD'][1]['converters']['target'] = float

Metrics_river = {'F1' : metrics.F1(), 'Accuracy' : metrics.Accuracy()}

results_river_multiclass = get_performance_river(Models_river, Datasets_river, Metrics_river)

River : Model ADWIN bagging and dataset CovtFD
River : Model Adaptive Random Forest and dataset CovtFD
River : Model Bagging and dataset CovtFD
River : Model Hoeffding Adaptive Tree and dataset CovtFD
River : Model Hoeffding Tree and dataset CovtFD
River : Model Leveraging Bagging and dataset CovtFD
River : Model Naive Bayes and dataset CovtFD
River : Model k-Nearest Neighbors and dataset CovtFD
River : Model SGDClassifier and dataset CovtFD


In [50]:
pd.DataFrame(results_river_multiclass)

,Model,Dataset,Time (seconds),F1,Accuracy
0,ADWIN bagging,CovtFD,237.9197,0.768907,0.732841
1,Adaptive Random Forest,CovtFD,643.9280,0.847260,0.809473
2,Bagging,CovtFD,197.9844,0.758322,0.719785
3,Hoeffding Adaptive Tree,CovtFD,598.7151,0.782246,0.724330
4,Hoeffding Tree,CovtFD,307.3757,0.727014,0.656878
5,Leveraging Bagging,CovtFD,956.8648,0.788018,0.755593
6,Naive Bayes,CovtFD,305.5696,0.136451,0.251982
7,k-Nearest Neighbors,CovtFD,240.1380,0.755216,0.715650
8,SGDClassifier,CovtFD,77.6858,0.762616,0.726390


The most notable with these multi-class classifiers are that the computational times are MUCH higher than before. That is because of the size of these datasets.

The computational times vary significantly between CapyMOA and River. CapyMOA generally requires much longer execution times for several models, including ADWIN Bagging, Adaptive Random Forest, Bagging, Leveraging Bagging, k-Nearest Neighbors, and SGDClassifier. For instance, ADWIN Bagging in CapyMOA takes 1156.58 seconds, compared to just 237.92 seconds in River, making River nearly five times faster for this model. Similarly, Adaptive Random Forest in CapyMOA requires 3117.22 seconds, whereas River completes the task in 643.93 seconds. However, River does not always outperform CapyMOA in terms of speed. For models such as Hoeffding Adaptive Tree, Hoeffding Tree, and Naive Bayes, River takes longer to compute. For example, Hoeffding Adaptive Tree runs in 39.34 seconds in CapyMOA but requires 598.72 seconds in River, making CapyMOA significantly faster for these specific models. This indicates that while River is generally faster for most models, CapyMOA can be more efficient for certain classifiers, particularly those based on simpler or more traditional algorithms.

In terms of performance, both frameworks deliver competitive results, with some notable exceptions. CapyMOA tends to achieve higher F1 and Accuracy scores across most models. For example, Adaptive Random Forest in CapyMOA achieves an F1 score of 0.87 and an Accuracy of 0.93, compared to 0.85 and 0.81, respectively, in River. Similarly, Leveraging Bagging in CapyMOA achieves an F1 score of 0.85 and an Accuracy of 0.92, outperforming River's 0.79 and 0.76. These results suggest that CapyMOA may offer superior predictive performance for many models, making it a preferable choice when accuracy is a priority.
The Naive Bayes model is an outlier, performing poorly in both frameworks, with particularly low Accuracy and F1 scores. This suggests that Naive Bayes may not be well-suited for the CovtFD dataset, regardless of the library used.

Overall, the choice between CapyMOA and River depends on the specific requirements of the task. If computational efficiency is a priority, River is generally the better choice for most models, except for Hoeffding Adaptive Tree, Hoeffding Tree, and Naive Bayes, where CapyMOA is faster. On the other hand, if predictive performance is the primary concern, CapyMOA tends to deliver better results across most models. 

## Part III - Regression

The regressors I found on both River and CapyMOA were : 
- Passive-Aggressive Regressor
- Stochastic Gradient Tree
- [baseline] Mean predictor
- k-Nearest Neighbors
- Adaptive Random Forest
  
We will compare both libraries on these regressors.

The datasets we will use are from CapyMOA and are the following : 
- Bike
- Fried

### III.2) Evaluating with CapyMOA

In [39]:
Models_capymoa = {'Passive-Aggressive Regressor': PassiveAggressiveRegressor,
         '[baseline] Mean predictor': TargetMean,
         'k-Nearest Neighbors': KNNRegressor,
         'Adaptative Random Forest': AdaptiveRandomForestRegressor}

results_capymoa_multiclass = get_performance_capymoa(Models_capymoa, {"Fried": Datasets["Fried"], "Bike": Datasets["Bike"]})


CapyMOA : Model Passive-Aggressive Regressor and dataset Fried


/opt/anaconda3/lib/python3.12/site-packages/capymoa/evaluation/evaluation.py:348: UserWarning: The learner did not produce a prediction for this instance
  warnings.warn("The learner did not produce a prediction for this instance")


CapyMOA : Model Passive-Aggressive Regressor and dataset Bike


/opt/anaconda3/lib/python3.12/site-packages/capymoa/evaluation/evaluation.py:348: UserWarning: The learner did not produce a prediction for this instance
  warnings.warn("The learner did not produce a prediction for this instance")


CapyMOA : Model [baseline] Mean predictor and dataset Fried
CapyMOA : Model [baseline] Mean predictor and dataset Bike
CapyMOA : Model k-Nearest Neighbors and dataset Fried
CapyMOA : Model k-Nearest Neighbors and dataset Bike
CapyMOA : Model Adaptative Random Forest and dataset Fried
CapyMOA : Model Adaptative Random Forest and dataset Bike


In [40]:
pd.DataFrame(results_capymoa_multiclass)

,Model,Dataset,Time (seconds),RMSE,MAE,R2
0,Passive-Aggressive Regressor,Fried,9.1796,3.556168,2.786693,0.493842
1,Passive-Aggressive Regressor,Bike,3.8511,103.982219,67.370865,0.671401
2,[baseline] Mean predictor,Fried,0.0771,4.998494,4.065592,0.000000
3,[baseline] Mean predictor,Bike,0.0613,181.395165,131.803362,0.000000
4,k-Nearest Neighbors,Fried,3.2969,2.875860,2.281269,0.668944
5,k-Nearest Neighbors,Bike,1.5328,127.939436,83.791926,0.502545
6,Adaptative Random Forest,Fried,48.6355,2.196962,1.720786,0.806818
7,Adaptative Random Forest,Bike,6.7185,97.432459,65.933862,0.711493


### III.3) Evaluating with River

In [17]:
PA_reg_mode1 = preprocessing.StandardScaler() | linear_model.PARegressor(mode=1, C=1.0)
PA_reg_mode2 = preprocessing.StandardScaler() | linear_model.PARegressor(mode=2, C=1.0)
SGT_reg = preprocessing.StandardScaler() | tree.SGTRegressor(max_depth=5)
Mean_reg = preprocessing.StandardScaler() | dummy.StatisticRegressor(stats.Mean())
KNN_reg = preprocessing.StandardScaler() | neighbors.KNNRegressor()
ARF_reg = (preprocessing.StandardScaler() | forest.ARFRegressor(seed=42))

In [35]:
Datasets_river = {'Fried' : ('Fried.csv', {'converters': {'0': float, '1': float, '2': float, '3': float, '4': float, '5': float, '6': float, '7': float, '8': float, '9': float, 'target':float}})
                  ,'Bike' : ('Bike.csv', {'converters': {'0': float, '1': float, '2': float, '3': float, '4': float, '5': float, '6': float, '7': float, '8': float, '9': float, '10': float, '11': float, 'target':float}})}

Models_river = {'Passive-Aggressive Regressor, mode 1': PA_reg_mode1,
         'Passive-Aggressive Regressor, mode 2': PA_reg_mode2,
         'Stochastic Gradient Tree': SGT_reg,
         '[baseline] Mean predictor': Mean_reg,
         'k-Nearest Neighbors': KNN_reg,
         'Adaptative Random Forest': ARF_reg}

Metrics_river = {'MAE' : metrics.MAE(), 'RMSE' : metrics.RMSE(), 'R2' : metrics.R2()}

Regressors_results_river = get_performance_river(Models_river, Datasets_river, Metrics_river)


River : Model Passive-Aggressive Regressor, mode 1 and dataset Fried
River : Model Passive-Aggressive Regressor, mode 1 and dataset Bike
River : Model Passive-Aggressive Regressor, mode 2 and dataset Fried
River : Model Passive-Aggressive Regressor, mode 2 and dataset Bike
River : Model Stochastic Gradient Tree and dataset Fried
River : Model Stochastic Gradient Tree and dataset Bike
River : Model [baseline] Mean predictor and dataset Fried
River : Model [baseline] Mean predictor and dataset Bike
River : Model k-Nearest Neighbors and dataset Fried
River : Model k-Nearest Neighbors and dataset Bike
River : Model Adaptative Random Forest and dataset Fried
River : Model Adaptative Random Forest and dataset Bike


In [36]:
pd.DataFrame(Regressors_results_river)

,Model,Dataset,Time (seconds),MAE,RMSE,R2
0,"Passive-Aggressive Regressor, mode 1",Fried,0.9119,5.966787,7.530535,-1.265627
1,"Passive-Aggressive Regressor, mode 1",Bike,0.4212,91.741340,137.123810,0.428477
2,"Passive-Aggressive Regressor, mode 2",Fried,0.8761,10.088037,12.609983,-5.352810
3,"Passive-Aggressive Regressor, mode 2",Bike,0.4191,181.742606,265.820005,-1.147746
4,Stochastic Gradient Tree,Fried,4.3603,2.586185,3.358342,0.549405
5,Stochastic Gradient Tree,Bike,0.9766,161.908217,236.186743,-0.695582
6,[baseline] Mean predictor,Fried,0.5542,4.058724,5.004636,-0.000649
7,[baseline] Mean predictor,Bike,0.2609,162.785361,233.538142,-0.657766
8,k-Nearest Neighbors,Fried,73.5031,2.176098,2.777278,0.691841
9,k-Nearest Neighbors,Bike,21.6292,81.180746,122.583749,0.543255


For the Passive-Aggressive Regressor, the River library has two modes. Because mode 1 had overall the best results, it is the one we will use to compare to CapyMOA. 

When it comes to execution time, River demonstrates a clear advantage with the Passive-Aggressive Regressor, executing approximately ten times faster than CapyMOA on both datasets. However, for the remaining models (Mean Predictor, k-Nearest Neighbors, and Adaptive Random Forest) CapyMOA consistently outperforms River. We think it is important to note that two methods were about 10x faster than the other two : Passive-Aggresive Regressor and Mean Predictor vs k-Nearest Random Forest and Adaptive Random Forest. So if we need a fast method, these would be the top choices. However, the performance of the mean predictor is not very good (R2 = 0).

In terms of performance metrics, CapyMOA is generally better. For the Passive-Aggressive Regressor, CapyMOA achieves superior RMSE, MAE, and R² values on both datasets, despite its slower execution time. The Mean Predictor also performs better in CapyMOA for the Fried dataset, while River is only slightly better in MAE for the Bike dataset. The k-Nearest Neighbors model is the only exception where River performs better than CapyMOA. For the Adaptive Random Forest, CapyMOA is better across all metrics, delivering significantly better RMSE, MAE, and R² values for both datasets.

In conclusion, for regressors, CapyMOA shows much better results, in terms of time and performance.

## Part IV - Anomaly Detection

The only anomaly detection method I found on both River and CapyMOA was HalfSpaceTrees.

We will compare both libraries on this classifier.

The datasets we will use are from CapyMOA and are the following : 
 - ElectricityTiny
 - RTG_2abrupt

### II.1) Training the model with CapyMOA

In [54]:
Models_capymoa = {'Half Space Trees': HalfSpaceTrees}

Datasets_capymoa = {k: Datasets[k] for k in ['elec_stream_tiny', 'RTG_2abrupt']}
results_capymoa = get_performance_capymoa(Models_capymoa, Datasets_capymoa)


CapyMOA : Model Half Space Trees and dataset elec_stream_tiny
CapyMOA : Model Half Space Trees and dataset RTG_2abrupt


In [55]:
pd.DataFrame(results_capymoa)[['Model', 'Dataset', 'Time (seconds)', 'Accuracy']]

,Model,Dataset,Time (seconds),Accuracy
0,Half Space Trees,elec_stream_tiny,0.2572,60.750
1,Half Space Trees,RTG_2abrupt,3.5943,22.177


In [58]:
Datasets_river = {'Electricity Tiny' : ('elec_stream_tiny.csv', {'converters': {str(i) : float for i in range (0, 6)}}),
                 'RTG_2abrupt' : ('RTG_2abrupt.csv', {'converters': {str(i) : float for i in range (0, 30)}})}

hst = (preprocessing.StandardScaler() | anomaly.HalfSpaceTrees(n_trees=5, height=3, window_size=3, seed=42))
Models_river = {'Half Space Trees': hst}
Metrics_river = {'Accuracy' : metrics.Accuracy()}

def get_performance_river_anomaly(Models, Datasets, Metric):
    results = []
    for model_name in Models.keys():
        model = Models[model_name].clone()
        for data_name in Datasets.keys():
            metrics = {name: metr.clone() for name, metr in Metric.items()}
            result_model={}
            beginning = time.time()   #XXX - put the time measurements at the same place
            print(f"River : Model {model_name} and dataset {data_name}")
            dataset_csv, params = Datasets[data_name]
            for x, y in stream.iter_csv(dataset_csv, target='target', **params):
                yp = model.score_one(x)  
                if yp is not None:
                    y01  = _to01(y)
                    yp01 = _to01(yp)

                    for metr in metrics.values():
                        metr.update(y01, yp01)

                model.learn_one(x, y) 
            end = time.time()
            
            for metric_name in metrics.keys():
                metr = metrics[metric_name]
                if 'Model' not in result_model.keys():
                    result_model = {'Model': model_name,
                                    'Dataset': data_name,
                                    'Time (seconds)': round(end - beginning, 4),
                                    metric_name: metr.get()}
                else :
                    result_model[metric_name] = metr.get()
            results.append(result_model)
    return results
    
results_river = get_performance_river_anomaly(Models_river, Datasets_river, Metrics_river)



River : Model Half Space Trees and dataset Electricity Tiny
River : Model Half Space Trees and dataset RTG_2abrupt


In [59]:
pd.DataFrame(results_river)

,Model,Dataset,Time (seconds),Accuracy
0,Half Space Trees,Electricity Tiny,0.0884,0.604302
1,Half Space Trees,RTG_2abrupt,5.0889,0.802088


The computational efficiency of the Half Space Trees model does not vary two significantly between River and CapyMOA. For the Electricity Tiny dataset, River demonstrates an advantage in speed, completing the algorithm in just 0.09 seconds, compared to 0.26 seconds for CapyMOA. This makes River nearly three times faster for this dataset.

However, for the RTG-2abrupt dataset, CapyMOA is faster than River, taking 3.6 seconds compared to 5.1 seconds for River. This indicates that CapyMOA may be more efficient for certain datasets. 

River is the better choice for accuracy, especially given its lead on the RTG-2abrupt dataset, while CapyMOA is only slightly better for Electricity Tiny.

Therefore, for anomaly detection, it seems that River is the better framework.

## Conclusion 

In the end, both CapyMOA and River perform well, but each in their own domain. It seem that for binary classification and regressors, CapyMOA has stronger results. For multi-class and anomaly detection, River is a stronger choice. This said, this depends on which model is being used and most importantly, it depends on the priority of the user : compromising accuracy for a speedier algorithm or having a better prediction even though it takes longer.
